In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from src.bayesian_network import BayesianNetwork

/Users/macbookair/Documents/bayesian_dataclass/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create synthetic data
np.random.seed(42)
n_samples = 1000

data = pd.DataFrame({
    'age': [45, 30, 60, 50],
    'gender': ['M', 'F', 'F', 'M'],
    'socioeconomic_status': ['middle', 'low', 'high', 'middle'],
    'location': ['urban', 'rural', 'urban', 'urban'],
    'stress_level': ['high', 'low', 'medium', 'high'],
    'diet_quality': ['average', 'poor', 'good', 'average'],
    'exercise': ['regular', 'none', 'occasional', 'regular'],
    'sleep_pattern': ['poor', 'good', 'average', 'poor'],
    'smoking': ['yes', 'no', 'no', 'yes'],
    'alcohol_consumption': ['moderate', 'none', 'high', 'moderate'],
    'fatigue': [1, 0, 1, 1],
    'fever': [1, 0, 1, 1],
    'rash': [0, 0, 1, 0],
    'previous_illnesses': ['none', 'asthma', 'diabetes', 'none'],
    'unknown_illness_A': [1, 0, 0, 1],
    'unknown_illness_B': [0, 1, 1, 0]
})

In [3]:
bn = BayesianNetwork(method='k2', max_parents=3, categorical_columns=['stress_level', 'diet_quality'])
bn.fit(data)

In [4]:
# Analyze the network structure
structure = bn.explain_structure()
print("Network structure:", structure)

# Create a function to analyze a patient
def analyze_patient(age, stress_level, diet_quality, symptoms):
    patient = pd.DataFrame({
        'age': [age],
        'stress_level': [stress_level],
        'diet_quality': [diet_quality],
        'fatigue': [1 if 'fatigue' in symptoms else 0],
        'fever': [1 if 'fever' in symptoms else 0],
        'rash': [1 if 'rash' in symptoms else 0]
    })
    
    illness_A_prob = bn.sample_node('unknown_illness_A', size=1000).mean()
    illness_B_prob = bn.sample_node('unknown_illness_B', size=1000).mean()
    
    sensitivity_A = bn.compute_sensitivity('unknown_illness_A')
    sensitivity_B = bn.compute_sensitivity('unknown_illness_B')
    
    return {
        'illness_A_probability': illness_A_prob,
        'illness_B_probability': illness_B_prob,
        'sensitivity_A': sensitivity_A,
        'sensitivity_B': sensitivity_B
    }

# Example usage
patient_analysis = analyze_patient(45, 'high', 'average', ['fatigue', 'fever'])
print("Patient analysis:", patient_analysis)

Network structure: {'age': [], 'stress_level': ['age'], 'diet_quality': ['age'], 'fatigue': ['age'], 'fever': ['age'], 'rash': ['age'], 'unknown_illness_A': ['age'], 'unknown_illness_B': ['age']}
Patient analysis: {'illness_A_probability': -0.3152652152016546, 'illness_B_probability': -1.9295821993795963, 'sensitivity_A': {'age': 0.657871733319209, 'stress_level': 0.4837814699104838, 'diet_quality': 0.4862129663185367, 'fatigue': 0.4836893131315003, 'fever': 0.4884747995870267, 'rash': 0.48736989504006806, 'unknown_illness_B': 0.4869557610506171}, 'sensitivity_B': {'age': 1.2761213866041436, 'stress_level': 0.9897594665431838, 'diet_quality': 1.001677395096315, 'fatigue': 0.9881040268209914, 'fever': 0.9983681368361375, 'rash': 0.9949741773230173, 'unknown_illness_A': 1.003297659070026}}
